<a href="https://colab.research.google.com/github/takky0330/Colab/blob/main/MovieLens100k_MF%EF%BC%88Matrix%20Factorization%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Matrix Factorizationを用いたレコメンド
今回はMovieLens100kを用いて実験。   
評価指標はRecall@10


In [1]:
from time import time
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

#### データセットの内容確認

In [2]:
u_data_org = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")
u_data_org.head()

,user_id,item_id,rating,timestamp
4,166,346,1,886397596


#### 学習用・テスト用データの取り込み

In [3]:
# ユーザ×評価値のデータ
u_data_train = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/ua.base', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")
u_data_test = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/ua.test', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")

# 件数の確認
train_cnt = u_data_train.count()
test_cnt = u_data_test.count()
print('Train Set:', str(train_cnt), '\n')
print('Test Set:', str(test_cnt))

Train Set: user_id      90570
item_id      90570
rating       90570
timestamp    90570
dtype: int64 

Test Set: user_id      9430
item_id      9430
rating       9430
timestamp    9430
dtype: int64


#### データをitem_id × user_idの行列へ整形
Qiitaの記事に従うとuser × itemが良いのですが以前から使っているのがitem_id × user_idなのでその形にしてあります。

In [6]:
# item_id x user_idの行列に変換する
item_list = u_data_org.sort_values('item_id').item_id.unique()
user_list = u_data_org.user_id.unique()
rating_matrix_item = np.zeros([len(item_list), len(user_list)])

for item_id in tqdm(range(1, len(item_list))):
    user_list_item = u_data_train[u_data_train['item_id'] == item_id].sort_values('user_id').user_id.unique()
    for user_id in user_list_item:
        try:
            user_rate = u_data_train[(u_data_train['item_id'] == item_id) & (u_data_train['user_id'] == user_id)].loc[:, 'rating']
        except:
            user_rate = 0
        rating_matrix_item[item_id-1, user_id-1] = user_rate


100%|██████████| 1681/1681 [02:28<00:00, 11.29it/s] 


In [11]:
rating_matrix_item.shape

(1682, 943)

In [12]:
# 本当はこんなオブジェクト使わなくてもいけると思うのですがいいやり方が浮かばず…。いい方法あれば教えてください。

# item x userの評価したかどうか{0, 1}がわかる行列作成
rating_matrix_calc = rating_matrix_item.copy()
rating_matrix_calc[rating_matrix_calc != 0] = 1
# 評価していないアイテムに1が立つ行列を作成。後で使う
rating_matrix_train = np.abs(rating_matrix_calc - 1)

### Matrix Factorization
ここからいよいよ行列分解していきます。

In [13]:
class MatrixFactorization():
  def __init__(self, R, X, Y, k, steps=200, alpha=0.01, lamda=0.001, threshold=0.001):
    self.R = R
    self.m = R.shape[0]
    self.n = R.shape[1]
    self.k = k
    # initializa U and V
    self.U = np.random.rand(self.m, self.k)
    self.V = np.random.rand(self.k, self.n)
    self.alpha = alpha
    self.lamda = lamda
    self.threshold = threshold
    self.steps = steps
    self.info_step = 1

    # preserve user_id list and item_id list
    self.X = X
    self.Y = Y

  def shuffle_in_unison_scary(self, a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

  def fit(self):
    for step in range(self.steps):
      start_time = time()
      error = 0
      # shuffle the order of the entry
      self.shuffle_in_unison_scary(self.X,self.Y)

      # update U and V
      for i in self.X:
        for j in self.Y:
          r_ij = self.R[i-1,j-1]
          if r_ij > 0:
            err_ij = r_ij - np.dot(self.U[i-1,:], self.V[:,j-1])
            for q in range(self.k):
              self.U[i-1,q] += self.alpha * (err_ij * self.V[q, j-1] + self.lamda * self.U[i-1, q])
              self.V[q, j-1] += self.alpha * (err_ij * self.U[i-1, q] + self.lamda * self.V[q, j-1])

      # approximation
      R_hat = np.dot(self.U, self.V)
      # calculate estimation error for observed values
      for i in self.X:
        for j in self.Y:
          r_ij = self.R[i-1, j-1]
          r_hat_ij = R_hat[i-1, j-1]
          if r_ij > 0:
            error += pow(r_ij - r_hat_ij,2)
      # regularization
      error += (self.lamda * np.power(self.U,2).sum()) / 2
      error += (self.lamda * np.power(self.V,2).sum()) / 2

      if step % self.info_step == 0 and step != 0:
        print('Step: %d / Error: %3f [%.1f sec]'%(step, error, time()-start_time))

      if error < self.threshold:
        break
    return self.U, self.V


In [14]:
X = u_data_org.item_id.unique()
Y = u_data_org.user_id.unique()
k = 20
steps = 150

mf = MatrixFactorization(rating_matrix_item, X, Y, k, steps)

In [ ]:
U, V = mf.fit()
pred_rating = np.dot(U, V)

In [19]:
pred_rating

array([[3.98590287, 3.56559202, 2.36669315, ..., 4.6081928 , 4.15996914,
        2.09074851],
       [3.93110095, 3.97524066, 2.00733114, ..., 3.17426514, 3.4548574 ,
        2.99509496],
       [4.31964347, 3.83933666, 2.94335065, ..., 2.5281691 , 3.33860187,
        0.23766795],
       ...,
       [4.31608793, 5.64012511, 3.13633536, ..., 3.74564034, 3.84568242,
        6.13557168],
       [5.35043158, 6.57456757, 2.94815603, ..., 3.72456979, 5.89629421,
        6.2984602 ],
       [5.68693954, 6.01020886, 5.7645435 , ..., 6.27332263, 6.26117799,
        3.79260687]])

### 予測評価値の計算・レコメンド

In [32]:
user_id = 1
hits = 0

# ユーザが既に評価したアイテムのスコアはゼロに直す
rating_matrix_user = rating_matrix_item[:, user_id - 1]
pred_rating_user_item = rating_matrix_user * rating_matrix_train[:,user_id - 1]

#ここからレコメンドされたアイテムがどれだけあっていたかを評価していく
recommend_list = np.argsort(pred_rating_user_item)[::-1][:10] + 1
purchase_list_user = u_data_test[u_data_test.user_id == user_id].loc[:, 'item_id'].unique()
for item_id in recommend_list:
    if item_id in purchase_list_user:
              hits += 1
              pre = hits / 10.0

              print(pred_rating_user_item)
              print('Recommend list:', recommend_list)
              print('Test Rated list:', purchase_list_user)
              print('Precision:', str(pre))
              hits += 1
pre = hits / 10.0

print(pred_rating_user_item)
print('Recommend list:', recommend_list)
print('Test Rated list:', purchase_list_user)
print('Precision:', str(pre))

[0. 0. 0. ... 0. 0. 0.]
Recommend list: [1682  578  554  555  556  557  558  559  560  561]
Test Rated list: [ 20  33  61 117 155 160 171 189 202 265]
Precision: 0.0


#### 全体の精度評価

In [17]:
# 予測評価値の計算
precision_list = []
recall_list = []
user_list_test = u_data_test.sort_values('user_id').user_id.unique()

for user_id in tqdm(user_list_test):
    hits = 0
    # ユーザが既に評価したアイテムのスコアはゼロに直す
    pred_rating_user_item = rating_matrix_user * rating_matrix_train[:,user_id - 1]

    #ここからレコメンドされたアイテムがどれだけあっていたかを評価していく
    recommend_list = np.argsort(pred_rating_user_item)[::-1][:10] + 1
    purchase_list_user = u_data_test[u_data_test.user_id == user_id].loc[:, 'item_id'].unique()
    for item_id in recommend_list:
        if item_id in purchase_list_user:
            hits += 1
    pre = hits / 10.0
    precision_list.append(pre)


100%|██████████| 943/943 [00:00<00:00, 1144.07it/s]


In [18]:
# 全体の精度検証
precision = sum(precision_list) / len(precision_list)
print('Precision:', precision)

Precision: 0.05121951219512226
